In [ ]:
!pip install pandas
!pip install numpy
!pip install scipy
!pip install sklearn

In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import gc
import sklearn

In [2]:
classifcols = pd.read_pickle('../data/classifcols.pkl')
regrcols = pd.read_pickle('../data/regrcols.pkl')
labencdist = pd.read_pickle('../data/labencdist.pkl')
labencsector = pd.read_pickle('../data/labencsector.pkl')
onehotenc = pd.read_pickle('../data/onehotenc.pkl')

In [10]:
totcols = list(set(classifcols).union(set(regrcols)).union(set(['Div_Paid?'])))
liabcols = ['Short-term debt','Accounts payable', 'Taxes payable',
            'Accrued liabilities','Other current liabilities', 
            'Total current liabilities','Long-term debt','Average Payables',
            'Deferred taxes liabilities','Other long-term liabilities', 
            'Total non-current liabilities','Total liabilities',
            'Days Payables Outstanding','Payable Turnover']
assetcols = ['Total cash','Receivables','Inventories','Prepaid expenses', 
             'Other current assets','Total current assets','Goodwill',
             'Gross property, plant and equipment','Other long-term assets',
             'Net property, plant and equipment','Intangible assets', 
             'Total non-current assets','Total assets','Average Inventory',
             'Average Receivables',"Total liabilities and stockholders' equity",
             'Cash per Share','Days Sales Outstanding','Days of Inventory on Hand',
             'Inventory Turnover']
inccols = ['Gross profit','Research and development','Operating income', 
           'Sales, General and administrative','Interest Expense',
           'Total operating expenses', 'Other income (expense)', 
           'Income before taxes','Provision for income taxes', 'Net income', 
           'Basic_EPS','Diluted_EPS', 'EBITDA', 'Interest Coverage',
           'Net Income per Share'] 
equitycols = ['Common stock', 'Additional paid-in capital','Retained earnings', 
              'Treasury stock','Accumulated other comprehensive income',
              'Market Cap', 'Book Value per Share','Invested Capital',
              "Shareholders Equity per Share"]
entvcols = ['EV to Free cash flow','Enterprise Value','stockprc','mth12stockratio',
            'mth6stockratio', 'PFCF ratio']
revcols = ['Revenue', 'Revenue per share']

def mdlinpt(comp):
    alldata = pd.read_pickle('../data/sp500finaldata.pkl')
    alldata.sort_values(by=['firm', 'Year'], ascending=[True, True], inplace=True)
    distenc = pd.DataFrame(labencdist.transform(alldata['DistanceFromLast']))
    sectenc = pd.DataFrame(labencsector.transform(alldata['Sector']))
    hotencoded = onehotenc.transform(pd.concat([sectenc, distenc], axis=1))
    namesdist = ['DistLast_'+i for i in labencdist.classes_]
    namessect = ['Sector_'+i for i in labencsector.classes_]
    names = np.append(namessect, namesdist)
    hotencoded = pd.DataFrame(hotencoded, columns=names)
    alldata.drop(['Sector', 'DistanceFromLast'], axis=1, inplace=True)
    alldata = pd.concat([alldata, hotencoded], axis=1)
    maxrec = alldata[(alldata.firm==comp) & (alldata.firm != alldata.firm.shift(-1))]
    maxrec = maxrec[totcols].reset_index(drop=True)
    return maxrec

def updater(maxrec, grrev, grinc, grasset, grliab, grentv):
    maxrec[assetcols] *= (1+ grasset/100)
    maxrec[liabcols] *= (1 + grliab/100)
    maxrec[inccols] *= (1 + grinc/100)
    maxrec[revcols] *= (1 + grrev/100)
    maxrec['Cost of revenue'] = maxrec['Revenue'] - maxrec['Gross profit']
    oldeq = maxrec["Total stockholders' equity"]
    maxrec["Total stockholders' equity"] = maxrec['Total assets'] - maxrec['Total liabilities']
    oldeq = maxrec["Total stockholders' equity"] / oldeq
    maxrec[equitycols] *= oldeq
    maxrec[entvcols] *= grentv
    #Updated final ratios
    maxrec['Intangibles to Total Assets']  = maxrec['Intangible assets'] / maxrec['Total assets']
    maxrec['Debt to Assets'] = maxrec['Total liabilities'] / maxrec['Total assets']
    maxrec['EV to Sales'] = maxrec['Enterprise Value'] / maxrec['Revenue']
    maxrec['Enterprise Value over EBITDA'] = maxrec['Enterprise Value'] / maxrec['EBITDA']
    maxrec['R&D to Revenue'] = maxrec['Research and development'] / maxrec['Revenue']
    maxrec['SG&A to Revenue'] = maxrec['Sales, General and administrative'] / maxrec['Revenue']
    maxrec['Price to Sales Ratio'] *= grentv/grrev
    maxrec['Paid_LastYr?'] = maxrec['Div_Paid?']
    maxrec['Net Debt to EBITDA'] = maxrec['Total liabilities'] / maxrec['EBITDA']
    maxrec['currentRatio'] = maxrec['Total current assets'] / maxrec['Total current liabilities']
    maxrec['grossProfitMargin'] = maxrec['Gross profit'] / maxrec['Revenue']
    maxrec['operatingProfitMargin'] = maxrec['Operating income'] / maxrec['Revenue']
    maxrec['assetTurnover'] = maxrec['Revenue'] / maxrec['Total assets']
    maxrec['returnOnAssets'] = maxrec['Net income'] / maxrec['Total assets']
    maxrec['Net Current Asset Value'] = maxrec['Total current assets'] - maxrec['Total liabilities']
    maxrec['PB ratio'] = maxrec['stockprc'] / maxrec['Book Value per Share']
    return maxrec

In [11]:
updater(mdlinpt('MSFT'),1,2,3,4,5)

KeyError: "['PBratio'] not in index"

In [8]:
alldata = pd.read_pickle('../data/sp500finaldata.pkl')
alldata['Net Current Asset Value']

0       4.281985e+09
1       3.770000e+08
2       8.700000e+07
3       5.780000e+08
4       4.281985e+09
5       4.281985e+09
6       4.281985e+09
7       4.281985e+09
8       4.281985e+09
9       4.281985e+09
10      2.884615e+09
11      3.597870e+08
12      5.926190e+08
13      1.815103e+10
14      6.177464e+10
15      4.227245e+10
16      3.583507e+10
17      3.592306e+10
18      4.281985e+09
19      4.281985e+09
20      4.281985e+09
21      4.281985e+09
22      4.281985e+09
23      4.281985e+09
24      4.281985e+09
25      4.586666e+09
26      4.281985e+09
27      6.554020e+09
28      2.593270e+08
29      3.056320e+09
            ...     
2062    4.281985e+09
2063    4.281985e+09
2064    4.281985e+09
2065    4.281985e+09
2066    4.281985e+09
2067    4.281985e+09
2068    4.281985e+09
2069    4.281985e+09
2070    4.281985e+09
2071    4.281985e+09
2072    4.281985e+09
2073    4.281985e+09
2074    4.281985e+09
2075    4.281985e+09
2076    4.281985e+09
2077    4.281985e+09
2078    4.281

In [9]:
alldata.columns.values

array(['Revenue', 'Cost of revenue', 'Gross profit',
       'Research and development', 'Sales, General and administrative',
       'Total operating expenses', 'Operating income', 'Interest Expense',
       'Other income (expense)', 'Income before taxes',
       'Provision for income taxes', 'Net income', 'Basic_EPS',
       'Diluted_EPS', 'EBITDA', 'Total cash', 'Receivables',
       'Inventories', 'Prepaid expenses', 'Other current assets',
       'Total current assets', 'Gross property, plant and equipment',
       'Accumulated Depreciation', 'Net property, plant and equipment',
       'Goodwill', 'Intangible assets', 'Other long-term assets',
       'Total non-current assets', 'Total assets', 'Short-term debt',
       'Accounts payable', 'Taxes payable', 'Accrued liabilities',
       'Other current liabilities', 'Total current liabilities',
       'Long-term debt', 'Deferred taxes liabilities',
       'Other long-term liabilities', 'Total non-current liabilities',
       'Total lia